In [49]:
import requests
import time
import re
import pandas as pd
import pprint
import pycrfsuite

import eli5
#import lime
#from lime.lime_text import LimeTextExplainer


from sklearn_crfsuite import CRF 
from sklearn.model_selection import cross_val_predict # train_test_split <- 쓰려면 쓰기
from sklearn_crfsuite .metrics import flat_classification_report # metrics: measure(Accuracy, score 등) 도구

from nltk.probability import FreqDist
from pycrfsuite_spacing import TemplateGenerator
from pycrfsuite_spacing import CharacterFeatureTransformer
from pycrfsuite_spacing import sent_to_xy
from bs4 import BeautifulSoup
from konlpy.tag import Kkma
from konlpy.tag import Okt
#from konlpy.tag import Mecab
#import Mecab
import tweepy


# score 정보 -> (원하는 것) 어떤 문장이 몇점인지를 예측 -> 이 값이 평균 이상이면 긍정, 평균 미만이면 부정
#            -> 긍/부정 으로만 학습하면 결과가 더 부정확할 수도. 
#            -> (현행) 평균 이상이면 100, 미만 0으로 분류해서 학습, 적용

# kkm 형태소 분석기에서 사전 조정해서, 칼퇴, 워라밸 같은 우리가 사용하는 단어로 수정하기.
# raw data : 맞춤법 교정.
# 조사 같은 연결어 제외하기. feature 추가(전전단어와 후후단어)
# CRF : N-gram ?????  kkm 사전을 형태소로 나눠놓고 하는 걸로. 

# 긍정/부정 리뷰 스크래핑+키워드 추출+사전으로 추가(feature) 
# 명사(를 비롯한 주요 단어)인 pos tag 만 뽑아서 feature에서 가중치 주기. 
# feature: pos tag에서 규칙 찾아서(eg. NNG+NNG) 여러 단어 연접하여 새롭게 긍/부정어 사전 돌리기 ??? <- 복잡. 
# 쪼개진 긍부정어 사전(만들어진 txt) 찾아보기.

# 조사 없애기? pos tag 포기. 띄어쓰기 교정 + 맞춤법 교정 + CRF ...? 
# feat -1, +1 < 최고로 나쁘다 -> 최고 로 나쁘 다 : 이런 식으로 CRF 특징이 무효화 됨.  

# CRF가 아니라, 다른 방식으로 해보는 게 어떤지? eg. review classification처럼 
# summarization + 빈도수 처리 + .... + N-gram 활용
# 시각화: eli5 

In [50]:
# 로그인 할 url
url = "https://www.jobplanet.co.kr/users/sign_in?_nav=gb"
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.97 Safari/537.36'
headers = {'Content-type': 'application/json', 'Accept': 'text/plain', 'User-Agent':user_agent}
login_data = {'user':{'email':'hsmy31@hanyang.ac.kr', 'password':'rhkwpgksmswnd!', 'remember_me':'true'}}
session = requests.session()

# 로그인 실행
login_response = session.post(url, json = login_data, headers = headers)

reviews = []

In [7]:
def ind_reviews(code):
    url = "https://www.jobplanet.co.kr/reviews?&industry_id=" + str(code)
    response =  session.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    num = soup.find('span', class_='num') # 리뷰 개수 확인
    num = int(num.get_text().strip())
    import math, random
    pages = math.ceil(num / 10)
    reviews = {}
    for i in range(pages+1): # 페이지를 랜덤으로 하여 300개의 리뷰 추출
        time.sleep(1)
        url = "https://www.jobplanet.co.kr/reviews?&industry_id=" + str(code) + "&page="+ str(i)
        response =  session.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        label = soup.find_all('h2', class_="us_label") # 한 줄 리뷰 추출
        # 추출한 리뷰에서 태그를 제외한 텍스트만 추출하여 labels 리스트에 저장
        labels = [label.get_text().strip() for label in label[:]]
        star = soup.find_all('div', class_="star_score") # 별점이 포함된 div 추출
        # 추출한 div에서 별점을 나타내는 width 속성의 숫자로 된 부분을 stars 리스트에 저장
        stars = re.findall('[0-9]+[.]+[0-9]', str(star))
        # reviews 딕셔너리에 '리뷰: 별점' 형식으로 추가
        for j in range(len(labels)):
            reviews[labels[j][5:-1]] = stars[j].replace('.0','')
    return reviews

In [32]:
review1=dict(ind_reviews(1001))

In [21]:
review2=dict(ind_reviews(900))

In [26]:
review2

{'많이 배울수 있는 기업이며, 인간 적인 기업 문화': '100',
 '발전중인게 체감되는 회사. IT를 접목함에 따라 충분히 발전할 수 있을 것 같습니다.': '100',
 '존중받으면서 일하고 워라밸이 좋은곳 좋은 사람들 좋은 상사': '100',
 '상호금융으로써 조합원 및 지역사회에 안정적인 경제활동을 도모하고 사회공헌활동으로 경제취약자 및 취약계층을 지원하는  기업': '100',
 '스타트업 기업으로서 투자 및 핀테크 개발업무를 주력으로 하며 회사의 성장가능성이 높고 수평적인 기업문화가 특징': '100',
 '업계1위의 안정적이고 좋은 회사입니다해당 분야의 커리어 쌓기 좋음': '100',
 '딜 많은 사모펀드로 여러 차례 굵직한 딜을 경험할 수 있음': '100',
 '제 인생을 바꿀 수 있게 해준 회사였습니다. 정말 최고의 회사였고, 앞으로도 무한한 성장이 가능하다 생각되는 회사입니다.': '96',
 'New York based 전 세계에서 가장 큰 자산운용사. 직원 수16,000 이상.': '94',
 'Top-tier 부동산전업운용사로 수탁고 기준 전세계 10위권. 국내에서 지난 5년간 수조원의 투자 유치로 성장세': '94',
 '합리적인 리더쉽, 보람 있는 일들을 할 수 있는 흔치 않은 조직': '94',
 '커리어를 쌓는데 최고.\r시간 떼우고 퇴근하는 직원들이라면.. 어딜가던 마찬가지겠지만\r스스로 공부하고 노력한다면 빠른 시간 내 승진기회 높음.': '94',
 '본인의 능력만큼 대우 받을 수 있는 곳. \r같이 일하는 직원들 다좋아요.': '94',
 '그래도 부동산운용사 중에서 직원으로 근무하기에는 제일 낫다고 봅니다': '94',
 'ICT 및 게임 분야 초기기업에 투자하는 벤처캐피탈리스트를 꿈꾼다면 대한민국 최고의 VC 중 하나': '92',
 '성장이 빠르고 가족 같은 분위기로 다양한 시장을 접근하는 투자 자문 회사': '92',
 '복지가 회사의 전부인 곳\r자신의 노력을 피할 수 없는 곳\r자유와 책임이 공존': '

In [27]:
review3=dict(ind_reviews(704))

In [28]:
review3

{'음 스타드업을 경험해보고 싶으신분 추천\r개발자라면 더더욱 추천': '100',
 '즐겁게 다닐 수 있었던 회사! 개발자가 많지 않아서 아쉽기도 했지만 전반적으로 가족같은 분위기': '100',
 '인턴으로서 매우 많은 것을 배울 수 있는 곳으로 만약 다른 사람이 이 회사의 인턴을 하게 된다면 매우 추천하고 싶다. 기초부터 튼튼하게 다질 수 있도록 도와주셔서 알고 있었던 것이라도 다시 한 번 짚고 갈 수 있도록 해준다.': '100',
 '상식이 통하는 스마트하고 좋은 사람들과 일할 수 있는 곳.\r당연하다고 생각했던 것들이 정말 당연한해서 오히려 신기한 곳.': '100',
 '다양한 개발환경을 경험할수 있고 여러가지 개발능력을 키울 수 있다.': '98',
 '좋은 회사입니다\r사장님이 착하시고 좋아요': '96',
 '피부과학에 독보적인 기술을 가지고 있고 시장 니즈가 있는 서비스를 하고 있습니다': '92',
 '가족같은 회사였다, 매우매우 많이 가족같은 회사였음.': '92',
 '만 20년이 넘은 한국 지사, 환경이 바뀌어도 꾸준히 극복하고 유지되며 회사가 돌아간다.': '90',
 '직원들끼리 우애 좋은 회사라서 다니기 편했습니다.': '90',
 '개발자가 본인의 가치와 퍼포먼스를 인정받으며 일할 수 있는 곳': '90',
 '엔지니어 우대 및 대우가 좋으며 교육을 장려함': '88',
 '마케팅 인턴으로 입사하여 근무했습니다. 직원들이 열심히 하는 회사라는 인상을 받았습니다.': '88',
 '복지와 기업 문화가 좋지만 프로세스가 불명확하고 시스템 체계를 잡는 게 빨리 필요함': '86',
 '개발자가 대우받으며 보람차게 일할 수 있는곳, 하지만 예전만 못한기업.': '86',
 '사내 분위기는 굉장히 좋으나 사업 추진력이 지지부진한 편.': '86',
 '여러갖로 나에게 도움을 준 경험을 준 회사이다. 실무적': '86',
 '성장하는 스타트업 경영진들이 젊고 사내 분위기도 수평적': '86',
 '서버, 클라이언트 개발 모두 경험할 수 

In [29]:
review4=dict(ind_reviews(1004))

In [30]:
review4

{'다양한 일을 배울 수 있고 자유로운 분위기인 편임.': '94',
 '기장팀이라 팀마다 다르겠지만 개인주의라면 자유롭고 편합니다': '90',
 '다양한 경험을 할 수 있고 문화 또하 좋았습니다.': '86',
 '작은곳이지만 마음만은 편한 회사였다 \r퇴사했어도 가끔 연락하고 좋았다': '86',
 '사람들이 너무 좋았고 좋은 상사를 만나서 많이 배울 수 있었음': '86',
 '결산, 조정 등 업무 배우기에 좋은 환경임': '86',
 '다양한 업무를 모두 경험할 수 있음 여의도에 위치해있어 출퇴근이 용이하고 식사나 커피 모두 다양하게 선택 가능함': '80',
 '지점이 여러개인데 지점끼리 서로 교류가 없음 지점마다 다르지만 일단 일이 많고 야근이 많음': '80',
 '전문지식을 배우며 업무를 빠르고 체계적으로 배울 수 있는 곳': '80',
 '내 할일만 잘 하면 되는 회사 걱정할건 없음': '80',
 '소규모이지만 업체를 다양하게 가지고 있어 여러 업종을 접할 수 있으며 수평적인 회사': '80',
 '업종이 세무대리쪽이여서 일하면서 여러가지 경력을 쌓을 수 있다는 최고의 장점이 있습니다. 이 업종은 어디서든 똑같은거 같지만, 회계사님들께서 회계학쪽으로 많이 도움 주십니다.': '80',
 '회계법인 중에서 최고의 대우를 받으며 일할 수 있는 곳.': '80',
 '사업부마다 다르겠지만, 제가 근무하고 있는 곳은 업무량도 많고 업무강도도 다소 높은 편입니다. 그에 비해 복지를 챙겨주려고 하는것 같긴 한데 업무량에 비하면 모자른것 같습니다.': '78',
 '신입으로 일배우기좋은곳이었어요 급여인상이나성취감에 집착하지않으면 좋은곳': '78',
 'A부터Z까지 모든걸 담당자가 직접 결산함 \r개인의 역량 강화를 위해서 자기개발이 필요함': '76',
 '첫직장으론 나쁘지않은곳.막내가 다니기엔 좋은회사지만,연차가 높아질수록 위에 사람이 나가지 않는이상 계속해서 막내생활을 함': '76',
 '좋은 분위기에서 일할수 있는곳, 발전하는 곳': '76',

In [33]:
print(len(review1),len(review2),len(review3),len(review4))

771 1061 342 374


In [34]:
#전체 리뷰 및 별점 딕셔너리화
codes = ['1001', '900', '704', '1004']
reviews={}

for i in range(0,len(codes)):
    reviews.update(dict(ind_reviews(codes[i])))

In [35]:
len(reviews)

2548

In [36]:
text1=list(reviews.keys())
score1=list(reviews.values())

In [38]:
#긍/부정 사전을 만들기 위한 평균값 산출
sum=0
for i in range(len(score1)):
    sum+=int(score1[i])
avr=sum/len(score1)
print("score 평균값: ", round(avr,3))

score 평균값:  56.203


In [81]:
#공통/ 업종별 불용어(자주 나오지만 문서 전체에 관련되어 있어 성능 체크에 의미가 없는 단어들) 선언
stopwords=["직장","회사","기업","업무","사람","직원","있음"]
stopwords1=["공공기관","공기업","공공","정부","부처","공무원","기관"]
stopwords2=["은행","금융","금융권","금융업","운용사","보험사","지점"]
stopwords3=["개발자","소프트웨어","IT"]
stopwords4=["세무","회계","회계사","세무사","지점","법인"]

In [82]:
#긍/부정 리뷰별 빈출단어 추출 및 사전화
#검색 성능을 높이기 위해 문장을 제일 잘게 쪼개는 kkma 사용

kkma=Kkma()

text_kkma=[]

#형태소 분석 및 문장별 점수 매칭

for i in range(len(text1)):
    text_kkma.append(kkma.nouns(text1[i]))
    text_kkma[i].append(score1[i])
    
print(text_kkma)

[['대체적', '근무', '환경', '부서', '업무량', '시', '야근', '필요', '경우', '존재', '100'], ['정년', '보장', '도서', '도서대출반납', '대출', '반납', '다양', '프로그램', '운영', '운영중', '중', '100'], ['대한', '대한민국', '민국', '식량', '식량산업', '산업', '최전선', '분투', '해군', '소수', '소수정예', '정예', '종자', '종자관리', '관리', '전문', '전문기관', '기관', '소속', '소속직원', '직원', '대한민국전체', '전체', '200', '200명', '명', '96'], ['수평적', '회사', '회사분위기', '분위기', '상사', '눈치', '안', '안보고', '보고', '휴가', '내가', '일', '외국인', '사람', '저녁', '삶', '가능', '직장', '96'], ['민원인', '업무', '업무강도', '강도', '조사관', '친절', '94'], ['세종', '세종시', '시', '활동', '기업', '기업지원기관', '지원', '기관', '신생', '신생기관', '94'], ['도청', '근무', '충청남', '충청남도', '도', '총괄적', '업무', '수행', '예하', '시', '시군의', '군의', '정책', '그림', '곳', '94'], ['서울', '도심', '국제기구', '국제', '국제현안', '현안', '수', '관련', '커리어', '있음', '94'], ['자유', '분위기', '보수적', '공공', '공공기관', '기관', '생각', '때', '느낌', '유사', '94'], ['노동법', '곳', '직원', '전반적', '함', '악성', '민', '민원인', '원인', '상대', '가나', '있음', '94'], ['워라벨', '연봉', '세금', '해외', '지역', '근무', '점', '단점', '힘', '92'], ['근무', '연가', '보', '보수가', '수가', '일', '만족

In [83]:
pos_kkma=[]
morphs_kkma=[]
dic_stopwords=stopwords1+stopwords2+stopwords3+stopwords4+stopwords
#연관이 낮아 보이는 형태소와 1글자 음절, 불용어를 제외한 체언, 용언, 관형사, 부사만 뽑아내기

for i in range(len(text1)):
    pos_kkma.append(kkma.pos(text1[i]))

dic_kkma=[]

for i in range(len(pos_kkma)):
    dic_kkma.append([])
    for j in range(len(pos_kkma[i])):
        if pos_kkma[i][j][0] not in dic_stopwords:
            if pos_kkma[i][j][1].startswith('N') or pos_kkma[i][j][1].startswith('V')or pos_kkma[i][j][1].startswith('M'):
                dic_kkma[i].append(pos_kkma[i][j][0])
    dic_kkma[i].append(int(score1[i]))


In [84]:
dic_kkma

[['대체적',
  '근무',
  '환경',
  '좋',
  '부서',
  '따르',
  '다르',
  '업무량',
  '몰리',
  '시',
  '야근',
  '필요',
  '경우',
  '존재',
  100],
 ['정년',
  '확실히',
  '보장',
  '도서',
  '대출',
  '반납',
  '아니',
  '다양',
  '프로그램',
  '운영',
  '중',
  '이',
  '좋',
  100],
 ['대한민국',
  '식량',
  '산업',
  '최전선',
  '분투',
  '해군',
  '같',
  '소수',
  '정예',
  '종자',
  '관리',
  '전문',
  '소속',
  '대한민국',
  '전체',
  '200',
  '명',
  '없',
  96],
 ['수평적',
  '이',
  '분위기',
  '좋',
  '상사',
  '눈치',
  '많이',
  '안',
  '보고',
  '휴가',
  '쓰',
  '내가',
  '하',
  '일',
  '열심히',
  '하',
  '되',
  '외국인',
  '일하',
  '것',
  '좋아하',
  '즐기',
  '저녁',
  '있',
  '삶',
  '가능',
  96],
 ['민원인', '많', '강도', '높', '조사관', '친절', 94],
 ['세종', '시', '활동', '좋', '지원', '이', '신생', '이', 94],
 ['도청',
  '근무',
  '충청남도',
  '관하',
  '총괄적',
  '이',
  '수행',
  '되',
  '예하',
  '시',
  '군의',
  '정책',
  '크',
  '그림',
  '그리',
  '곳',
  94],
 ['서울',
  '도심',
  '있',
  '국제기구',
  '국제',
  '현안',
  '다루',
  '수',
  '있',
  '관련',
  '커리어',
  '쌓',
  '수',
  94],
 ['자유', '분위기', '보수적', '이', '않', '상당히', '생각', '때', '느낌', '가장', '유사', 9

In [85]:
# 긍/부정 빈도수 체크하여 상위 50개를 사전으로 저장

dic_text_pos=[]
dic_text_neg=[]  

for i in range(len(dic_kkma)):
    for j in range(0, len(dic_kkma[i])):
        if type(dic_kkma[i][j]) != int:
            if len(dic_kkma[i][j])>1:
                if float(dic_kkma[i][ -1]) > avr:
                    dic_text_pos.append(dic_kkma[i][j])
                else:
                    dic_text_neg.append(dic_kkma[i][j])

fdist1=FreqDist(dic_text_pos)
dic_text_pos=fdist1.most_common(50)

fdist1=FreqDist(dic_text_neg)
dic_text_neg=fdist1.most_common(50)

In [86]:
dic_text_pos1=[]
dic_text_neg1=[]

for i in range(len(dic_text_pos)):
    dic_text_pos1.append(dic_text_pos[i][0])
    
for i in range(len(dic_text_neg)):
    dic_text_neg1.append(dic_text_neg[i][0])

print(dic_text_pos1)
print(dic_text_neg1)

['일하', '분위기', '다니', '근무', '경험', '다양', '배우', '문화', '복지', '성장', '환경', '생각', '가지', '연봉', '편하', '추천', '대하', '안정적', '많이', '가능', '아니', '급여', '조직', '최고', '발전', '계약', '괜찮', '자유', '워라밸', '업계', '시간', '대우', '강도', '매우', '나쁘', '여러', '하지만', '보장', '힘들', '사업', '따르', '수평적', '워라벨', '경력', '부서', '본인', '노력', '개발', '비하', '신입']
['다니', '분위기', '일하', '배우', '생각', '추천', '문화', '다양', '복지', '힘들', '너무', '아니', '경험', '경력', '연봉', '많이', '근무', '신입', '체계', '사무실', '급여', '대하', '괜찮', '그냥', '대표', '자유', '성장', '편하', '환경', '나쁘', '발전', '시간', '매우', '위하', '못하', '개인', '모르', '따르', '바뀌', '다르', '안정적', '계약', '가지', '사업', '가능', '이상', '보수적', '보험', '조직', '오래']
